In [194]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow import keras

In [195]:
# import temperature data
data = pd.read_csv('temp/temperature.csv').values
data = np.delete(data,0,1)
print(data)
n_steps = 4

[[27.37 23.29]
 [28.77 20.87]
 [28.73 22.41]
 [28.13 20.35]
 [28.37 19.5 ]
 [29.07 17.68]
 [28.53 19.27]
 [27.7  19.13]
 [27.07 20.2 ]
 [28.27 21.39]
 [28.13 20.72]
 [27.33 20.3 ]
 [27.1  21.56]
 [28.53 22.15]
 [28.33 23.65]
 [27.6  24.13]
 [28.3  23.85]
 [28.77 22.79]
 [29.03 24.22]
 [27.6  23.43]
 [28.07 24.24]
 [28.63 23.02]
 [27.9  19.6 ]
 [27.57 21.4 ]
 [27.07 20.8 ]
 [28.47 22.6 ]
 [28.17 22.63]
 [27.93 22.63]]


In [214]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [233]:
n_features=2
# convert into input/output
X, y = split_sequences(data[:20], n_steps)
xtest, ytest = split_sequences(data[20:], n_steps)
# print(data[20:23])
for i in range(len(xtest)):
	print(xtest[i], ytest[i])
# X.shape
data.shape

[[28.07 24.24]
 [28.63 23.02]
 [27.9  19.6 ]
 [27.57 21.4 ]] [27.07 20.8 ]
[[28.63 23.02]
 [27.9  19.6 ]
 [27.57 21.4 ]
 [27.07 20.8 ]] [28.47 22.6 ]
[[27.9  19.6 ]
 [27.57 21.4 ]
 [27.07 20.8 ]
 [28.47 22.6 ]] [28.17 22.63]
[[27.57 21.4 ]
 [27.07 20.8 ]
 [28.47 22.6 ]
 [28.17 22.63]] [27.93 22.63]


(28, 2)

In [229]:
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features) ))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')

In [230]:
# fit model
model.fit(X, y, epochs=1000, verbose=0)
# demonstrate prediction


In [231]:
yhat = model.predict(xtest, verbose=0)
print(yhat)
print(ytest)


[[28.508886 17.485611]
 [28.511578 21.153353]
 [28.144075 23.457577]
 [27.727978 22.862825]]
[[27.07 20.8 ]
 [28.47 22.6 ]
 [28.17 22.63]
 [27.93 22.63]]


In [232]:
newModel = Sequential()
newModel.add(LSTM(100,return_sequences=True,activation='relu',stateful=True,batch_input_shape=(1,None,2)))
newModel.add(LSTM(100,return_sequences=True,activation='relu',stateful=True))
newModel.add(Dense(n_features))
newModel.compile(optimizer='adam', loss='mse')
newModel.set_weights(model.get_weights())

In [ ]:
lastSteps = np.empty((28,-16,2)) #includes a shift at the beginning to cover the gap 
lastSteps[:,:4] = X[:,-4:] #the initial shift steps are filled with x training data 
newModel.predict(X[:,:-4,:]).reshape(1,1,2) #just to adjust states, predict with x without the last shift elements

rangeLen = totalLength-trainLength-shift
print('rangeLen: ', rangeLen)
for i in range(rangeLen):
    lastSteps[:,i+shift] = newModel.predict(lastSteps[:,i:i+1,:]).reshape(1,1,2)
print(lastSteps.shape)
forecastFromSelf = lastSteps[:,shift:,:]
print(forecastFromSelf.shape)